<img src="logos/‎cover‎001.png" alt="Drawing"/>

<img src="logos/‎redbar001.png" alt="Drawing"/>
<br />

## **Overview**
1. Ecological validity and VR
2. EEG and VR setup
3. Applications
4. Latency tests
5. ERP stimulus onsets

<img src="logos/‎redbar001.png" alt="Drawing"/>
<br />

# 1. Ecological validity and VR

### **EV**:
- how well findings achieved within non-naturalistic and action-deprived contexts **generalize** to the real-world conditions in which the brain evolved (Nastase et al., 2020; Voelkl et al., 2020).
- Findings that fail to **generalize** may lead to misconceptions about human cognitive behavior Kingstone et al. (2003)
- Reevaluate conventional methods and develop experiments resembling real-world conditions.
- Real-world neuroscience is a difficult task and most methods are unsuitable.
- CAUTION: setup should capture important variables, no recreate real life (Schmuckler, 2001; Chang et al., 2022).

<img src="logos/‎redbar001.png" alt="Drawing"/>
<br />

# 1. Ecological validity and VR

### **VR**:
- _"Synthetic sensory information that leads to perceptions of environments and their contents as if they were not synthetic."_ Blascovich et al. (2002, p. 105)
- Simulated environments are perceptual and auditory and can allow for movement and action
- _“computer-generated world”_ (Pan and Hamilton, 2018)
-